# Passwords

## Introduction

This manual is going to illustrate how I have created a secure CLI password manager that is cloud based using Python and a MariaDB database hosted on a DigitalOcean droplet. The password manager will allow you to store, retrieve, and update passwords in a secure and encrypted manner. It will also allow you to generate strong passwords for you to use.

## Project overview

The project is divided into three main parts:

```text
1. The CLI password manager
2. The database schema
3. The database connection
```

### 1. The CLI password manager

The CLI password manager is a Python script that allows you to store, retrieve, and update passwords in a secure and encrypted manner. It uses the Fernet encryption method to encrypt and decrypt passwords before storing them in the database. The Fernet encryption method is a symmetric encryption method that uses a shared secret key to encrypt and decrypt data.

**First you'll need a encryption key. I created a small script to generate one.**

```python
from cryptography.fernet import Fernet

def generate_key():
    # Generate a key
    key = Fernet.generate_key()

    # Display the key
    print(f"Your encryption key: {key.decode()}")
    print(
        "Write this key down somewhere safe. You'll need it to encrypt and decrypt your data."
    )

if __name__ == "__main__":
    generate_key()
```

**Pseudo-code for the passwords program:**

```text
1. Import necessary libraries for encryption, database connection, and SSH tunneling

2. Define SSH and database connection parameters

3. Function to load encryption key from user input

4. Function to open SSH tunnel and connect to remote server

5. Function to connect to MySQL database

6. Function to disconnect from MySQL database

7. Function to close SSH tunnel

8. Function to run SQL queries and return results as a pandas DataFrame

9. Application logic: 
   * a. Display application banner
   * b. Load encryption key from user input
   * c. Validate the encryption key
   * d. Open SSH tunnel and connect to MariaDB database
   * e. Enter main loop:

10. Execute the main function
```

### The main loop:

With the while loop we can execute a set of statements as long as a condition is true. In this case, we want to keep looping until the user enters the exit command. We can use the input function to get user input and store it in the choice variable. If the user enters exit, we break out of the loop and exit the program.

```text
* i. Prompt user for command: [store], [generate], [retrieve], [update], or [exit]

* ii. If [store]:
    - Prompt user for account description, username, and password
    - Encrypt password and store in database

* iii. If [generate]:
    - Generate a random password and display it

* iv. If [retrieve]:
    - Retrieve and decrypt all stored passwords, display them

* v. If [update]:
    - Prompt user for ID of password to update
    - Update and re-encrypt the password in the database

* vi. If [exit]:
    - Exit the application

* f. Disconnect from the database and close SSH tunnel
```

**Here's the snippet I used to generate unique passwords:**

```python
def generate_password(length=12):
    alphabet = string.ascii_letters + string.digits + string.punctuation
    password = "".join(secrets.choice(alphabet) for i in range(length))
    return password

print(f"\nGenerated password: {generate_password()}")
```

### 2) The database schema

```sql
CREATE TABLE passwords (
    id INTEGER PRIMARY KEY,
    description TEXT NOT NULL,
    username TEXT NOT NULL,
    password TEXT NOT NULL
);
```

The SQL commands I used in my code were:

```sql
-- Storing passwords
INSERT INTO passwords (id, description, username, password)
VALUES (%s, %s, %s, %s);

-- Retrieving passwords
SELECT * FROM passwords;

-- Updating passwords
UPDATE passwords SET description=%s, username=%s, password=%s WHERE id=%s;
```

To execute any SQL commands, I use the `run_query` function which utilizes Pandas to run the SQL query and return the results.

```python
def run_query(sql):
    return pd.read_sql_query(sql, connection)
```

### 3) The database connection

I used 127.0.0.1 as the host in my database connection because it refers to the local loopback interface, which allows the application to connect to services running on the same machine. In this case, the database is hosted on the SSH server, and by establishing an SSH tunnel, your code effectively forwards requests from your local machine to the database server’s local address. This setup ensures a secure connection while allowing you to access the database as if it were running locally on your own machine.

Let's walk through the logic I implemented to connect to the database and execute queries.

1. **SSH Tunnel**: 
   - First, we create an SSH tunnel to securely connect to the remote database server using the `SSHTunnelForwarder`.
   - This is done in the `open_ssh_tunnel` function, which starts the tunnel.

2. **MySQL Connection**:
   - After the SSH tunnel is established, we connect to the MariaDB database with `pymysql.connect()` in the `mysql_connect` function.
   - We use `localhost` and the local bind port provided by the SSH tunnel.

3. **Query Execution**:
   - The `run_query` function executes SQL queries using `pandas` to easily handle the results.
   - We perform operations like storing, retrieving, and updating passwords using SQL commands.

4. **Cleanup**:
   - Finally, we close the database connection with `mysql_disconnect()` and the SSH tunnel with `close_ssh_tunnel()` when we're done.

```python
import pymysql
from sshtunnel import SSHTunnelForwarder

# SSH and database connection parameters
ssh_host = "your_ssh_host"
ssh_port = 22
ssh_username = "your_ssh_username"
ssh_password = "your_ssh_password"
database_username = "your_db_username"
database_password = "your_db_password"
database_name = "your_db_name"

def open_ssh_tunnel():
    tunnel = SSHTunnelForwarder(
        (ssh_host, ssh_port),
        ssh_username=ssh_username,
        ssh_password=ssh_password,
        remote_bind_address=("127.0.0.1", 3306),
    )
    tunnel.start()
    return tunnel

def mysql_connect(tunnel):
    connection = pymysql.connect(
        host="127.0.0.1",
        user=database_username,
        passwd=database_password,
        db=database_name,
        port=tunnel.local_bind_port,
    )
    return connection

def mysql_disconnect(connection):
    connection.close()

def close_ssh_tunnel(tunnel):
    tunnel.close()

# Example usage
tunnel = open_ssh_tunnel()
connection = mysql_connect(tunnel)

# Perform database operations here...

mysql_disconnect(connection)
close_ssh_tunnel(tunnel)

```

> #### If you'd like to use the code, read it or work with you can check out the [Github repository](https://github.com/john-u/passwords) for the project. Also, I wrote an article on [Relational Databases](https://john-u.com/relational-databases/) if you're interested in learning more about databases.